In [22]:
import shutil
import sys
import os
from pathlib import Path
import json
import re

In [23]:
root_path = Path('/home/jianghui/dataset/Hierachal-MovieNet/')
img_root_path = Path(root_path, 'all/img/')
shot_root_path = Path(root_path, 'all/shot/')
scene_root_path = Path(root_path, 'all/scene/')
subtitle_root_path = Path(root_path, 'all/subtitle/')
synopsis_root_path = Path(root_path, 'all/synopsis/')
cast_root_path = Path(root_path, 'all/cast/')
subtitle_img_root_path = Path(root_path, 'all/subtitle_img/')
names = locals()

In [24]:
dest_path = Path('/home/jianghui/dataset/VideoReorder-MovieNet/')
dest_all_path = Path(dest_path, 'all/')

In [25]:
tt_list = []
with open(Path(root_path, 'movie_id_list'), 'r') as f_tt_list:
    for tt_line in f_tt_list:
        tt_list.append(tt_line.replace('\n', ''))

In [26]:
def open_json(types:str, tt_id):
    names[f'{types}_path'] = Path(names[f'{types}_root_path'], f'{tt_id}.json')
    with open(names[f'{types}_path'], 'r') as names[f'f_{types}']:
        names[f'{types}_json'] = json.load(names[f'f_{types}'])
    return names[f'{types}_json']

def save_json(types:str, tt_id):
    names[f'{types}_path'] = Path(names[f'{types}_root_path'], f'{tt_id}.json')
    with open(names[f'{types}_path'], 'w') as names[f'f_{types}']:
        json.dump(names[f'{types}_json'], names[f'f_{types}'], indent=2)

In [46]:
first = re.compile('\d+(?=_)')
second = re.compile('(?=_)\d+')

In [58]:
DEBUG = True

img_group_idx = 0 # the folder name id
total_group = 0

for tt_id in (tt_list if not DEBUG else ['tt0056923']):
    print(f'processing {tt_id} ...')
    shot_json = open_json('shot', tt_id)
    subtitle_img_json = open_json('subtitle_img', tt_id)

    # gain img select in shot
    img_subtitle_paired = []
    for subtitle_img_json_ele in subtitle_img_json:
        if subtitle_img_json_ele['sentence'] == [] or subtitle_img_json_ele['sentence'] == [""]:
            img_subtitle_paired.append(0)
        else:
            img_subtitle_paired.append(1)

    if DEBUG: print(img_subtitle_paired)

    img_num = 0
    img_list = []
    img_list_ele = []
    old_img_num = len(subtitle_img_json)
    rate_boundary = 0.8

    i = 0 # 10 <= j -i + 1 <= 20
    while i < old_img_num - 9:
        
        j = i + 9
        tmp_rate = sum(img_subtitle_paired[i: (j+1)]) / 10.0
        if tmp_rate < rate_boundary:
            i += 1
            continue

        j += 1
        last_j = 0
        while j < old_img_num:

            tmp_rate = sum(img_subtitle_paired[i:(j+1)]) / ((j+1-i))
            if tmp_rate < rate_boundary:
                last_j = j - 1
                break
            
            if j+1-i > 20:
                last_j = j - 1
                break

            j += 1

        if last_j == 0: last_j = old_img_num - 1

        img_list_ele = [i, last_j]
        img_list.append(img_list_ele)
        
        i = j

    if DEBUG: print("img_list is \n",img_list, "\n", "len is", len(img_list))
    total_group += len(img_list)
    assert False

    for idx, img_list_ele in enumerate(img_list):
        file_name = img_group_idx
        img_group_idx += 1
        file_path = Path(dest_all_path, str(file_name))
        Path.mkdir(file_path, exist_ok=True)

        img_num = (img_list_ele[1] + 1 - img_list_ele[0])

        # generate img
        for idx, old_img_id in enumerate(range(img_list_ele[0], img_list_ele[1] + 1)):
            old_img_name = subtitle_img_json[old_img_id]['img_id'] + ".jpg"
            old_img_path = Path(img_root_path, tt_id, old_img_name)
            new_img_name = f'{file_name}_{idx}.jpg'
            new_img_path = Path(file_path, new_img_name)
            shutil.copyfile(old_img_path, new_img_path)

        # generate and save subtitle.json
        new_subtitle_json = [[] for i in range(img_num)]
        for img_id_idx, img_id in enumerate(range(img_list_ele[0], img_list_ele[1] + 1)):
            new_subtitle_json[img_id_idx] = subtitle_img_json[img_id]["sentence"]

        new_subtitle_file_path = Path(file_path, "subtitle.json")
        with open(new_subtitle_file_path, 'w') as f_new_subtitle_file:
            json.dump(new_subtitle_json, f_new_subtitle_file, indent=2)
        
        # generate and save info.json
        info_json = {}
        info_json["tt_id"] = tt_id
        info_json["img_num"] = img_num
        info_json["img_id"] = [idx for idx in range(img_num)]
        info_json["shot_id"] = []
        info_json["scene_id"] = []
        for img_id in range(img_list_ele[0], img_list_ele[1] + 1):
            img_name = subtitle_img_json[img_id]["img_id"]
            shot_id = int(first.search(img_name).group(0))
            info_json["shot_id"].append(shot_id)
            scene_id = shot_json[shot_id]["scene_id"]
            info_json["scene_id"].append(scene_id)


        info_json_path = Path(file_path, "info.json")
        with open(info_json_path, 'w') as f_info_json:
            json.dump(info_json, f_info_json, indent=2)

processing tt0056923 ...
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,

AssertionError: 

In [57]:
# statistic
shot_sum_list = [0 for i in range(20)]
scene_sum_list = [0 for i in range(20)]
for id_path in dest_all_path.iterdir():
    info_path = Path(id_path, "info.json")
    with open(info_path, 'r') as f_info:
        info_json = json.load(f_info)
    shot_num = len(set(info_json['shot_id']))
    scene_num = len(set(info_json['scene_id']))
    shot_sum_list[shot_num] += 1
    scene_sum_list[scene_num] += 1

for i in range(20):
    print(f'{shot_sum_list[i]} data have {i} shot, {scene_sum_list[i]} data have {i} scene')



0 data have 0 shot, 0 data have 0 scene
0 data have 1 shot, 6833 data have 1 scene
0 data have 2 shot, 1975 data have 2 scene
0 data have 3 shot, 722 data have 3 scene
390 data have 4 shot, 282 data have 4 scene
1304 data have 5 shot, 113 data have 5 scene
1732 data have 6 shot, 52 data have 6 scene
1647 data have 7 shot, 24 data have 7 scene
1288 data have 8 shot, 10 data have 8 scene
937 data have 9 shot, 8 data have 9 scene
767 data have 10 shot, 5 data have 10 scene
626 data have 11 shot, 2 data have 11 scene
457 data have 12 shot, 1 data have 12 scene
326 data have 13 shot, 1 data have 13 scene
245 data have 14 shot, 2 data have 14 scene
169 data have 15 shot, 1 data have 15 scene
74 data have 16 shot, 0 data have 16 scene
48 data have 17 shot, 0 data have 17 scene
17 data have 18 shot, 0 data have 18 scene
4 data have 19 shot, 0 data have 19 scene
